In [1]:
import pandas as pd
import numpy as np

In [2]:
twitter_data = pd.read_csv('twitter_stemmed_data.csv', encoding = 'ISO-8859-1')

In [3]:
twitter_data.head()

,Target,Ids,Date,Flag,User,Text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [4]:
twitter_data['Text'][0]

"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D"

In [5]:
twitter_data.shape

(1600000, 7)

In [6]:
twitter_data.isnull().sum()

Target               0
Ids                  0
Date                 0
Flag                 0
User                 0
Text                 0
stemmed_content    495
dtype: int64

In [7]:
twitter_data.dropna(inplace = True)

In [8]:
twitter_data['Target'].value_counts()

1    799766
0    799739
Name: Target, dtype: int64

# Training Model 

In [9]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ksonu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
X = twitter_data['stemmed_content'].values
Y = twitter_data['Target'].values

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify = Y, test_size = 0.2, random_state = 2)

In [13]:
print(X_train)
print(X_test)
print(Y_train)
print(Y_test)

['retrorewind great show today' 'still wide awak good' 'raphi hai dear'
 ... 'buildabear yeah tri yet kinda came oven n thank love seri much'
 'feel well today im sick hate flu even breath well urgghh'
 'read http bit ly ckw cool know guy chat']
['wow rain day sick' 'watch pineappl express' 'josh shep mate' ...
 'love day back work tomorrw thou' 'papercak might throw traffic'
 'adzyuk might nce cheap addit em look like major airtim']
[1 0 1 ... 1 0 1]
[0 1 1 ... 0 1 1]


# convert text data into numerical data 

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

vectorizer = TfidfVectorizer()

X_train_num = vectorizer.fit_transform(X_train)
X_test_num = vectorizer.transform(X_test)

# Save the fitted vectorizer
with open('tfidf_vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)


In [15]:
model = LogisticRegression(max_iter = 1000)

In [16]:
model.fit(X_train_num, Y_train)

LogisticRegression(max_iter=1000)

In [17]:
X_train_prediction = model.predict(X_train_num)
train_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print(train_data_accuracy)

0.8101568922885518


In [18]:
X_test_prediction = model.predict(X_test_num)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print(test_data_accuracy)

0.7787284191046605


In [19]:
import pickle
filename = 'pred_trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [20]:
my_model = pickle.load(open('pred_trained_model.sav','rb'))

In [21]:
n = int(input('Enter the index value for predict: '))
print("Tweet : ", twitter_data['text'][n])
X_new = X_test_num[n]
print(Y_test[n])
prediction = my_model.predict(X_new)
accuracy = accuracy_score(Y_test, X_test_prediction)
if(prediction == 0):
    print("negative Tweet")
else:
    print('Positive Tweet')
print(prediction)
print(accuracy*100,'%')

ValueError: invalid literal for int() with base 10: ''